[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/Planejamento_Agregado/2_PlanejamentoAgregado_pyomo.ipynb)

# Modelagem e Gestão de Processos

TEP - UFF


**Professor: Diogo Ferreira de Lima Silva**

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [1]:
# Importando o Pyomo
import pyomo.environ as pe 
import pandas as pd

## Exercício - Planejamento de um Evento

O organizador de uma conferência deve fornecer almoços para os cinco dias de uma conferência. O organizador se depara com o problema de decidir sobre o fornecimento diário de guardanapos limpos. Ele sabe que os números de participantes da conferência que precisam almoçar nos cinco dias são 130, 220, 180, 120 e 100. Suas alternativas são: comprar novos guardanapos a 25 centavos cada. Enviar guardanapos usados para a lavandaria onde podem receber um serviço de 48 horas a 10 centavos o guardanapo, ou um serviço de 24 horas a 15 centavos o guardanapo. Considere que esta conferência é uma atividade única, não será repetida no próximo mês/ano. Como o organizador deve fornecer guardanapos para minimizar o custo total?


In [13]:
# Definindo o modelo
M = pe.ConcreteModel()

# Conjuntos
Dias = [1,2,3,4,5]

# Parâmetros
Demandas = {1: 130, 2: 220, 3: 180, 4: 120, 5: 100}
C_novo = 0.25
C_48 = 0.10
C_24 = 0.15

# Variáveis de decisão
x = M.x = pe.Var(Dias, within=pe.NonNegativeIntegers)
y48 = M.y48 = pe.Var(Dias, within=pe.NonNegativeIntegers)
y24 = M.y24 = pe.Var(Dias, within=pe.NonNegativeIntegers)
s = M.s = pe.Var(Dias, within=pe.NonNegativeIntegers) #estoque no final do dia 


# Função objetivo
M.custo_total = pe.Objective(
    expr=sum(C_novo * x[i] for i in Dias) +
         sum(C_48 * y48[i] for i in Dias if i <= 4) + 
         sum(C_24 * y24[i] for i in Dias),
    sense=pe.minimize
)

# Restrições
M.constraints = pe.ConstraintList()

# Restrições de demanda diária
M.constraints.add(x[1] >= Demandas[1])
M.constraints.add(x[2] + y24[1] >= Demandas[2])
M.constraints.add(x[3] + y48[1] + y24[2] >= Demandas[3])
M.constraints.add(x[4] + y48[2] + y24[3] >= Demandas[4])
M.constraints.add(x[5] + y48[3] + y24[4] >= Demandas[5])

# Restrições de disponibilidade dos guardanapos lavados em 48 horas
M.constraints.add(y24[1]+y48[1] <= x[1])
M.constraints.add(y24[2]+y48[2] <= x[2] + y24[1])
M.constraints.add(y24[3]+y48[3] <= x[3] + y24[2])
M.constraints.add(y24[4]+y48[4] <= x[4] + y24[3])


# Solucionando o modelo
solver = pe.SolverFactory('glpk')
solver.solve(M)


{'Problem': [{'Name': 'unknown', 'Lower bound': 127.5, 'Upper bound': 127.5, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 15, 'Number of nonzeros': 28, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.21642017364501953}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
# Imprimindo os resultados
print("Resultados:")

print(f"Demandas = {Demandas}")
for i in Dias:
    print(f"Dia {i}:")
    print(f"  Guardanapos novos comprados: {pe.value(x[i])}")
    if i>=2:
        print(f" ---------------")
        print(f"  Guardanapos recebidos da lavanderia (24h): {pe.value(y24[i-1])}")
        if i>=3:
            print(f"  Guardanapos recebidos da lavanderia (48h): {pe.value(y48[i-2])}")

    if i <= 4:
        print(f" ---------------")
        print(f"  Guardanapos enviados para lavagem (24h): {pe.value(y24[i])}")
        if i <= 3:
            print(f"  Guardanapos enviados para lavagem (48h): {pe.value(y48[i])}")
        
    print(f" ---------------")
print("Custo total = ", pe.value(M.custo_total))

Resultados:
Demandas = {1: 130, 2: 220, 3: 180, 4: 120, 5: 100}
Dia 1:
  Guardanapos novos comprados: 130.0
 ---------------
  Guardanapos enviados para lavagem (24h): 130.0
  Guardanapos enviados para lavagem (48h): 0.0
 ---------------
Dia 2:
  Guardanapos novos comprados: 90.0
 ---------------
  Guardanapos recebidos da lavanderia (24h): 130.0
 ---------------
  Guardanapos enviados para lavagem (24h): 180.0
  Guardanapos enviados para lavagem (48h): 40.0
 ---------------
Dia 3:
  Guardanapos novos comprados: 0.0
 ---------------
  Guardanapos recebidos da lavanderia (24h): 180.0
  Guardanapos recebidos da lavanderia (48h): 0.0
 ---------------
  Guardanapos enviados para lavagem (24h): 80.0
  Guardanapos enviados para lavagem (48h): 100.0
 ---------------
Dia 4:
  Guardanapos novos comprados: 0.0
 ---------------
  Guardanapos recebidos da lavanderia (24h): 80.0
  Guardanapos recebidos da lavanderia (48h): 40.0
 ---------------
  Guardanapos enviados para lavagem (24h): 0.0
 ------

## Um Problema de Planejamento Agregado

Vamos relembrar o problema de planejamento agregado visto em sala. 

Um estudo de previsão de demanda realizado em uma fábrica resultou nos seguintes resultados para os próximos 4 meses: 52400, 45800, 65000 e 87600.

A produção pode ser realizada de maneira regular, a partir de horas extras e/ou com o uso de subcontratação de terceiros, com custos unitários respectivamente de: 480,00, 600,00 e 750,00. Além disso, o custo unitário de manter em estoque por um período é de 225,00 unidades monetárias.

As capacidades para cada período são ilustradas na tabela a seguir:

| Período | Demandas | Capacidade Regular | Capacidade com Horas Extras | Capacidade com Subcontratações |
|---------|----------|---------------------|-----------------------------|-------------------------------|
| 1       | 52400    | 52400               | 8400                        | 4200                          |
| 2       | 45800    | 42900               | 8400                        | 4200                          |
| 3       | 65000    | 62300               | 8400                        | 4200                          |
| 4       | 87600    | 62300               | 8400                        | 4200                          |


Sabe-se que existe um estoque inicial de 5000 peças e deseja-se concluir o último período com 8000 peças em estoque. Qual o planejamento ideal para a produção das peças (quanto produzir em cada período com cada tipo de produção)? 



## Etapa 1: Coleta de Dados do Problema

In [ ]:
periodos = [1,2,3,4]
decisoes = ['Regular', 'HoraExtra', 'Sub']

In [ ]:
capacidades = pd.DataFrame({'Regular': [52400, 42900, 62300, 62300],
                        'HoraExtra': [8400 for i in range(4)],
                        'Sub': [4200 for i in range(4)]
                        }, index = periodos)
capacidades

In [ ]:
capacidades_dict = capacidades.stack().to_dict()
capacidades_dict

In [ ]:
custos = {'Regular': 480, 'HoraExtra': 600, 'Sub': 750}
custos

In [ ]:
demandas = {1: 52400, 2: 45800, 3: 65000, 4: 87600}
demandas

## Etapa 2: Modelagem

### Criando o modelo

Para instanciar o modelo, basta usar a função ConcreteModel():

In [ ]:
M = pe.ConcreteModel()

### Conjuntos

Criando os nossos conjuntos:
- Um conjunto para os períodos, chamado de I
- Um conjunto para os tipos de produção, chamado de J

In [ ]:
M.I = pe.Set(initialize = periodos)
M.J = pe.Set(initialize = decisoes)

### Parâmetros

Nossos parâmetros incluem as demandas, capacidades de produção e os custos

In [ ]:
M.demandas = pe.Param(M.I, initialize = demandas)
M.capacidades = pe.Param(M.I, M.J, initialize =  capacidades_dict)
M.custos_prod = pe.Param(M.J, initialize = custos)
M.custo_manter = pe.Param(initialize = 225)

### Variáveis de Decisão

In [ ]:
M.x = pe.Var(M.I, M.J, within = pe.NonNegativeReals)
M.estoque_inicial = pe.Var(M.I, within = pe.NonNegativeReals)
M.estoque_final = pe.Var(M.I, within = pe.NonNegativeReals)

### Função Objetivo

In [ ]:
def custo_total(M):
    return sum(M.x[i,j] * M.custos_prod[j] for i in M.I for j in M.J) + sum(M.estoque_final[i] * M.custo_manter for i in M.I)

M.z = pe.Objective(rule = custo_total, sense = pe.minimize)

### Restrições

In [ ]:
# Restrições de demanda
M.R_demanda = pe.ConstraintList()
for i in M.I:
    M.R_demanda.add(expr = M.estoque_inicial[i] + sum(M.x[i, j] for j in M.J) >= M.demandas[i])

# Restrições de capacidade
M.R_capacidade = pe.ConstraintList()
for i in M.I:
    for j in M.J:
        M.R_capacidade.add(expr = M.x[i,j] <= M.capacidades[i,j])

# Lógica dos Estoques
M.R_estoquefinal = pe.ConstraintList()
M.R_estoqueinicial = pe.ConstraintList()
for i in M.I:    
    M.R_estoquefinal.add(expr = M.estoque_inicial[i] + sum(M.x[i, j] for j in M.J) - M.demandas[i] == M.estoque_final[i])
    if i > 1: 
        M.R_estoqueinicial.add( expr = M.estoque_inicial[i] == M.estoque_final[i-1]) # Estoque inicial [i] == estoque final [i-1]

# Estoque inicial do primeiro período deve ser 5000
M.R_estoquenicial_1 = pe.Constraint(expr = M.estoque_inicial[1] == 5000)

# Estoque final do último período deve ser 8000
M.estoque_final_n = pe.Constraint(expr = M.estoque_final[4] == 8000)

### Resolvendo o Modelo e obtendo a solução alcançada

Para resolver o modelo, precisamos de um solver. Usaremos aqui o glpk

In [ ]:
# RESOLUÇÃO DO MODELO
pe.SolverFactory('glpk').solve(M)

In [ ]:
print(f"z= {pe.value(M.z)}")

for k in M.x.keys(): 
    if pe.value(M.x[k]) > 0:
        print (f"x_{k} = {pe.value(M.x[k])}")


for k in M.estoque_inicial.keys(): 
    if pe.value(M.estoque_inicial[k]) > 0:
        print (f"estoque_inicial_{k} = {pe.value(M.estoque_inicial[k])}")

for k in M.estoque_final.keys(): 
    if pe.value(M.estoque_final[k]) > 0:
        print (f"estoque_final_{k} = {pe.value(M.estoque_final[k])}")

## Exercício

Uma organização acaba de concluir sua previsão de demanda para o próximo ano. 

| Período | Demandas |
|---------|----------|
| Jan       | 1100    |
| Fev       | 1200    |
| Mar       | 1200    |
| Abr       | 1500    |
| Mai       | 1600    |
| Jun       | 1400    |
| Jul       | 1700    |
| Ago       | 1800    | 
| Set       | 2000    | 
| Out       | 2300    | 
| Nov       | 1800    | 
| Dez       | 1600    | 


No momento, 16 funcionários são responsáveis pela produção. Sabe-se que cada funcionário consegue entregar 100 unidades do produto por mês de forma regular e 10 unidades a partir de horas extras. Além disso, considera-se que a capacidade regular de produção da fábrica varia linearmente com o aumento/dimuição do número de funcionários. 

Os seguintes valores foram estimados pela equipe responsável pela gestão de custos e são constantes para todos os meses:

- Custo de contratação: R$ 1000/colaborador
- Custo de demissão: R$ 3500/colaborador
- Custo de estocar: R$ 5,00/produto/mês
- Custo regular de produção: R$ 20,00/produto
- Custo de produção com horas extras: R$ 24,00/produto

Sabe-se que, inicialmente, não há unidades em estoque. Além disso, os gestores gostariam que, ao fim do último mês, não fique unidades em estoque para o ano seguinte.

Considerando as informações mencionadas acima. Resolva o problema usando as seguintes estratégias:

1. Utilizando força de trabalho constante (não é permitida a contratação e demissão de funcionários). O seu plano fez uso de horas extras? Usou estoques?
2. Digamos que aconteceu um acidente no armazem. Assim, estoques não serão permitidos. Determine o plano agregado considerando também que agora você pode variar a força de trabalho.
3. Considere que a legislação vigente não permite ociosidade na força de trabalho (se a demanda de um período for menor do que a força de trabalho, você precisará obrigatoriamente demitir). O resultado obtido no plano agregado 2 seria alterado?
4. Formule agora o plano considerando que você possa usar qualquer estratégia que quiser (contratação e subcontratação, estoques, horas extras) e que não há legislação sobre a ociosidade na força de trabalho.